# Лекция 5. Машинное обучение, линейная регрессия

In [ ]:
from IPython.display import Image
import pandas as pd #pip install pandas/openpyxl
import plotly.express as px #pip install plotly
import statsmodels.formula.api as smf #pip install statsmodels
import re
from sklearn.model_selection import train_test_split #pip install scikit-learn
from sklearn.linear_model import LinearRegression
import plotly.graph_objects as go
import numpy as np #pip install numpy

In [43]:
Image(url='https://wiki.loginom.ru/images/crisp.svg')

Машинное обучение заключается в извлечении знаний из данных. Это
научная область, находящаяся на пересечении статистики,
искусственного интеллекта и компьютерных наук и также известная как
прогнозная аналитика или статистическое обучение. В последние годы
применение методов машинного обучения в повседневной жизни стало
обыденным явлением. Многие современные веб-сайты и устройства
используют алгоритмы машинного обучения, начиная с автоматических
рекомендаций по просмотру фильмов, заказа еды или покупки
продуктов, и заканчивая персонализированными онлайн-
радиотрансляциями и распознаванием друзей на фотографиях. 
Когда вы видите сложный сайт типа Facebook, Amazon или Netflix, то весьма
вероятно, что каждый раздел сайта содержит несколько моделей
машинного обучения.

Для решения задач с помощью машинного обучения (Machine Learning, ML):
* Данные — примеры решений и всё, что может помочь в процессе обучения: статистика, примеры текстов, расчеты, показатели, исторические события. Данные собирают годами и объединяют в огромные массивы — датасеты, которые есть у всех ИТ-корпораций. Примером сбора является капча, которая просит вас выбрать все фото с автомобилями и запоминает правильные ответы;
* Признаки — они же свойства или характеристики. Это то, на что должна обратить внимание машина в процессе обучения. Например, цена акций, изображение животного, частотность слов или пол человека. Чем меньше признаков и чем четче они обозначены и оформлены, тем проще обучаться. Однако для сложных задач современным моделям приходится учитывать десятки миллионов параметров, определяющих, как входы преобразуются в выходы;
* Алгоритмы — это способ решения задачи. Для одной и той же задачи их может быть множество и важно выбрать самый точный и эффективный.

In [44]:
Image(url='https://s0.rbk.ru/v6_top_pics/resized/1200xH/media/img/7/25/756237507753257.jpeg')

In [45]:
Image(url='https://s0.rbk.ru/v6_top_pics/resized/590xH/media/img/0/69/756237500508690.jpeg')

## Линейная регрессия

Если хотим предсказать значение НЕПРЕРЫВНОЙ ПЕРЕМЕННОЙ - решаем задачу регрессии

In [55]:
#сразу практика
laptops=pd.read_csv('data\laptop_price.csv',encoding='Windows-1251')
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37kg,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86kg,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83kg,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37kg,1803.60


In [47]:
laptops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         1303 non-null   int64  
 1   Company           1303 non-null   object 
 2   Product           1303 non-null   object 
 3   TypeName          1303 non-null   object 
 4   Inches            1303 non-null   float64
 5   ScreenResolution  1303 non-null   object 
 6   Cpu               1303 non-null   object 
 7   Ram               1303 non-null   object 
 8   Memory            1303 non-null   object 
 9   Gpu               1303 non-null   object 
 10  OpSys             1303 non-null   object 
 11  Weight            1303 non-null   object 
 12  Price_euros       1303 non-null   float64
dtypes: float64(2), int64(1), object(10)
memory usage: 132.5+ KB


Результирующая переменная - Price_euros. Будем предсказывать цену ноутбука по его характеристикам.

In [56]:
#попробуем предсказать значение цены по весу ноутбука
laptops['Weight']=laptops['Weight'].str.replace('kg','').astype('float64')
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60


In [49]:
import plotly.express as px
px.scatter(laptops,x='Weight',y='Price_euros')

In [ ]:
#уравнение прямой
#как работает линейная регрессия
#метод наименьших квадратов

In [50]:
#реализация модели с помощью statsmodels
model = smf.ols(formula='Price_euros ~ Weight', data=laptops).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Price_euros   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.044
Method:                 Least Squares   F-statistic:                     60.24
Date:                Fri, 03 Mar 2023   Prob (F-statistic):           1.69e-14
Time:                        15:53:14   Log-Likelihood:                -10353.
No. Observations:                1303   AIC:                         2.071e+04
Df Residuals:                    1301   BIC:                         2.072e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    673.1866     61.054     11.026      0.0

In [ ]:
#уравнение на выходе
y=221x+673

In [51]:
#многофакторная регрессия
model = smf.ols(formula='Price_euros ~ Weight + Inches', data=laptops).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Price_euros   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     56.42
Date:                Fri, 03 Mar 2023   Prob (F-statistic):           3.19e-24
Time:                        15:56:19   Log-Likelihood:                -10328.
No. Observations:                1303   AIC:                         2.066e+04
Df Residuals:                    1300   BIC:                         2.068e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   2551.6493    271.537      9.397      0.0

In [ ]:
y=513x1-165x2+2552

## Реализация линейной регрессии в scikit-learn

In [57]:
#для начала предобработаем данные
#на основе Memory выделим три признака по типу памяти SSD, HDD, Flash Storage
ssd=[]
hdd=[]
fs=[]
for i in laptops['Memory']:
    if 'SSD' in i:
        ssd.append(1)
        hdd.append(0)
        fs.append(0)
    elif 'HDD' in i:
        ssd.append(0)
        hdd.append(1)
        fs.append(0)
    else:
        ssd.append(0)
        hdd.append(0)
        fs.append(1)
laptops['ssd']=ssd
laptops['hdd']=hdd
laptops['fs']=fs
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ssd,hdd,fs
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128GB SSD,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1,0,0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34,898.94,0,0,1
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256GB SSD,Intel HD Graphics 620,No OS,1.86,575.00,1,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512GB SSD,AMD Radeon Pro 455,macOS,1.83,2537.45,1,0,0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256GB SSD,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1,0,0


In [58]:
#от памяти оставим только объем
s = [float(re.findall(r'\d+', s)[0]) for s in laptops['Memory']]
laptops['Memory']=s
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ssd,hdd,fs
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1,0,0
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128.0,Intel HD Graphics 6000,macOS,1.34,898.94,0,0,1
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256.0,Intel HD Graphics 620,No OS,1.86,575.00,1,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512.0,AMD Radeon Pro 455,macOS,1.83,2537.45,1,0,0
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1,0,0


In [79]:
ult=[]
for t in laptops['TypeName']:
    if t=='Ultrabook':
        ult.append(1)
    else:
        ult.append(0)
laptops['ult']=ult

In [97]:
laptops.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_euros,ssd,hdd,fs,ult
0,1,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 2.3GHz,8GB,128.0,Intel Iris Plus Graphics 640,macOS,1.37,1339.69,1,0,0,1
1,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128.0,Intel HD Graphics 6000,macOS,1.34,898.94,0,0,1,1
2,3,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,256.0,Intel HD Graphics 620,No OS,1.86,575.00,1,0,0,0
3,4,Apple,MacBook Pro,Ultrabook,15.4,IPS Panel Retina Display 2880x1800,Intel Core i7 2.7GHz,16GB,512.0,AMD Radeon Pro 455,macOS,1.83,2537.45,1,0,0,1
4,5,Apple,MacBook Pro,Ultrabook,13.3,IPS Panel Retina Display 2560x1600,Intel Core i5 3.1GHz,8GB,256.0,Intel Iris Plus Graphics 650,macOS,1.37,1803.60,1,0,0,1


In [98]:
laptops.columns

Index(['laptop_ID', 'Company', 'Product', 'TypeName', 'Inches',
       'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight',
       'Price_euros', 'ssd', 'hdd', 'fs', 'ult'],
      dtype='object')

In [99]:
laptops_model=laptops[['Inches','Memory','Weight','ssd','hdd','fs','ult','Price_euros']]
laptops_model.head()

,Inches,Memory,Weight,ssd,hdd,fs,ult,Price_euros
0,13.3,128.0,1.37,1,0,0,1,1339.69
1,13.3,128.0,1.34,0,0,1,1,898.94
2,15.6,256.0,1.86,1,0,0,0,575.00
3,15.4,512.0,1.83,1,0,0,1,2537.45
4,13.3,256.0,1.37,1,0,0,1,1803.60


In [88]:
laptops_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1303 entries, 0 to 1302
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Inches       1303 non-null   float64
 1   Memory       1303 non-null   float64
 2   Weight       1303 non-null   float64
 3   ssd          1303 non-null   int64  
 4   hdd          1303 non-null   int64  
 5   fs           1303 non-null   int64  
 6   ult          1303 non-null   int64  
 7   Price_euros  1303 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 81.6 KB


In [100]:
#делим на на признаки и результирующиюся
X=laptops_model.drop(['Price_euros'],axis=1)
Y=laptops_model['Price_euros']

In [101]:
X

,Inches,Memory,Weight,ssd,hdd,fs,ult
0,13.3,128.0,1.37,1,0,0,1
1,13.3,128.0,1.34,0,0,1,1
2,15.6,256.0,1.86,1,0,0,0
3,15.4,512.0,1.83,1,0,0,1
4,13.3,256.0,1.37,1,0,0,1
...,...,...,...,...,...,...,...
1298,14.0,128.0,1.80,1,0,0,0
1299,13.3,512.0,1.30,1,0,0,0
1300,14.0,64.0,1.50,0,0,1,0
1301,15.6,1.0,2.19,0,1,0,0


In [102]:
#разбиение на обучающую и тестовую выборку
#test_size - объем обучающей выборки
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

* Обучающая выборка - на ней обучаем нашу модель
* Тестовая - проверяем, как хорошо наша модель обучилась

In [103]:
regressor = LinearRegression() # а вот и наша модель
regressor.fit(X_train, y_train) # А тут обучение на обучающей выборке

LinearRegression()

In [104]:
#свободный коэф
print(regressor.intercept_)

1270.2751758832678


In [105]:
print(regressor.coef_)

[-1.16874522e+02  5.77623725e-01  5.95330158e+02  3.65463894e+02
 -1.97658207e+02 -1.67805686e+02  5.39551417e+02]


## Как проверить качество нашей модели?

In [106]:
#для начала предскажем с помощью нашей модели цену для тестовых данных
y_pred=regressor.predict(X_test)
y_pred[:10]

array([ 881.20845299, 1338.3455695 , 1564.72022454,  645.5003855 ,
       1334.62000273,  547.77179735,  788.3796235 ,  420.509059  ,
       1210.56153544, 1904.87008587])

In [107]:
true_vs_pred = pd.DataFrame({'Истинные': y_test, 'Предсказанные': y_pred})
true_vs_pred

,Истинные,Предсказанные
580,389.00,881.208453
1017,2968.00,1338.345570
761,1859.00,1564.720225
686,612.61,645.500385
186,2397.00,1334.620003
...,...,...
1176,825.00,660.970541
549,1729.00,1292.946892
922,1149.00,1596.239940
399,2625.00,1212.347884


In [74]:
random_x = np.linspace(0, len(true_vs_pred['Истинные']))

fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Истинные'],
                    mode='lines',
                    name='истинные'))
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Предсказанные'],
                    mode='lines+markers',
                    name='предсказанные'))

fig.show()

In [96]:
random_x = np.linspace(0, len(true_vs_pred['Истинные']))

fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Истинные'],
                    mode='lines',
                    name='истинные'))
fig.add_trace(go.Scatter(x=random_x, y=true_vs_pred['Предсказанные'],
                    mode='lines+markers',
                    name='предсказанные'))

fig.show()

In [108]:
model = smf.ols(formula='Price_euros ~ Weight + Inches + Memory + ssd + hdd + fs + ult', data=laptops_model).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            Price_euros   R-squared:                       0.424
Model:                            OLS   Adj. R-squared:                  0.421
Method:                 Least Squares   F-statistic:                     158.8
Date:                Fri, 03 Mar 2023   Prob (F-statistic):          3.17e-151
Time:                        16:23:34   Log-Likelihood:                -10024.
No. Observations:                1303   AIC:                         2.006e+04
Df Residuals:                    1296   BIC:                         2.010e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    815.8716    177.044      4.608      0.0

In [ ]:
y = 479*weight - 122*inches + 0.7*memory + 837*ssd + 166*hdd + 199*fs + 1202

In [78]:
y = 479*2.5-122*15.6+0.7*1024+837+1202
print(y*70)

143507.0


In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test,y_pred)
print('Коэффициент детерминации:',r2)